# Stock Market Strategy Prediction

## Import Dataset

In [ ]:
import numpy as np
import pandas as pd
train = pd.read_csv('/train.csv',index_col = 0)
test = pd.read_csv('/test.csv',index_col = 0)

## Data Visualization

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train.head()

In [ ]:
df = test.copy()

In [ ]:
print(df.isnull().sum())

In [ ]:
df2 = train.copy()

In [ ]:
print(df2.isnull().sum())

In [ ]:
print(df.dtypes)
print(df2.dtypes)

In [ ]:
plt.hist(df['Open'])
plt.show()

In [ ]:
plt.hist(df['Volume'])
plt.show()

In [ ]:
plt.hist(df2['Open'])
plt.show()

In [ ]:
plt.hist(df2['Volume'])
plt.show()

In [ ]:
print(df2['Open'].describe())
print(df2['Volume'].describe())
print(df2['Close'].describe())

In [ ]:
print(df2['Open'].corr(df2['Close']))
print(df2['Open'].corr(df2['Volume']))
print(df2['Open'].corr(df2['Volume']))

In [ ]:
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics

## Autocorrelation Plot

In [ ]:
from pandas.plotting import autocorrelation_plot
autocorrelation_plot(df2['Close'])
autocorrelation_plot(df2['Open'])
autocorrelation_plot(df2['Volume'])
plt.show()

## Scatter Matrix

In [ ]:
from pandas.plotting import scatter_matrix
scatter_matrix(df2[['Open' , 'Close' , 'Volume']], alpha=0.2, figsize=(6, 6))
plt.show()

print(df2['Close'].skew())
print(df2['Close'].kurtosis())

In [ ]:
print(df2['Open'].skew())
print(df2['Open'].kurtosis())

In [ ]:
print(df2['Volume'].skew())
print(df2['Volume'].kurtosis())